In [2]:
import yaml
from story_sage.utils.embedding import load_chunk_from_disk
import glob
from story_sage.utils.local_entity_extractor import StorySageEntityExtractor
from story_sage.story_sage_entity import StorySageEntityCollection


In [3]:
with open('config.yml', 'r') as file:
    config = yaml.safe_load(file)

api_key = config['OPENAI_API_KEY']
chroma_path = config['CHROMA_PATH']
chroma_collection = config['CHROMA_COLLECTION']
series_path = config['SERIES_PATH']

TARGET_SERIES_ID = 3 # wheel of time
TARGET_BOOK_NUMBER = 1

In [4]:
# Load series.yml to create a mapping from series_metadata_name to series_id
with open(series_path, 'r') as file:
    series_list = yaml.safe_load(file)

target_series_info = next(series for series in series_list if series['series_id'] == TARGET_SERIES_ID)

series_metadata_name = target_series_info['series_metadata_name']

In [4]:
chunks_path = f'./chunks/{series_metadata_name}/semantic_chunks/{TARGET_BOOK_NUMBER}_*.pkl'

chunks = []

for chunk_path in glob.glob(chunks_path):
    chunks.extend(load_chunk_from_disk(chunk_path))

In [6]:

extractor = StorySageEntityExtractor(series = target_series_info, device='mps')

entity_collection: StorySageEntityCollection = extractor.get_grouped_entities(chunks)

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/Users/chris.patten/.pyenv/versions/3.11.4/envs/story_sage_env/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


Extracting entities from chunks:   0%|          | 0/2362 [00:00<?, ?it/s]

Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


In [7]:
extractor = StorySageEntityExtractor(series = target_series_info, device='mps')

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

/Users/chris.patten/.pyenv/versions/3.11.4/envs/story_sage_env/lib/python3.11/site-packages/transformers/convert_slow_tokenizer.py:561: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(


In [5]:
with open(f'./entities/{series_metadata_name}/entities.json', 'r') as file:
    entity_collection = StorySageEntityCollection.from_json(file.read())

In [8]:
extractor.entity_collection = entity_collection

In [9]:
extractor.regroup_entities(new_threshold=0.5)

Regrouping entities with threshold 0.5:   0%|          | 0/1012 [00:00<?, ?it/s]

In [10]:
entity_collection = extractor.entity_collection

In [12]:
entity_json = entity_collection.to_json()
with open(f'./entities/{series_metadata_name}/entities.json', 'w') as file:
    file.write(entity_json)